In [3]:
#import libraries
!pip install kaggle
!pip install pandas
!pip install mysql-connector-python
import kaggle
!kaggle competitions download nfl-big-data-bowl-2025

nfl-big-data-bowl-2025.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
#downlaod dataset using kaggle api 
!kaggle competitions download -c nfl-big-data-bowl-2025

nfl-big-data-bowl-2025.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
#extract file from zip file 
import zipfile
import os

# Define the zip file name and extraction directory
zip_file = 'nfl-big-data-bowl-2025.zip'
extract_dir = 'nfl_data'

# Extract all the files
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_dir) # extract file to directory
zip_ref.close() # close file

In [6]:
#verify .cvs file in zip folder
import os

# List all files in the extracted directory
extracted_files = os.listdir(extract_dir)
print(extracted_files)

['games.csv', 'players.csv', 'player_play.csv', 'plays.csv', 'tracking_week_1.csv', 'tracking_week_2.csv', 'tracking_week_3.csv', 'tracking_week_4.csv', 'tracking_week_5.csv', 'tracking_week_6.csv', 'tracking_week_7.csv', 'tracking_week_8.csv', 'tracking_week_9.csv']


In [7]:
#read data from the file and handle null values 
import pandas as pd
import os

# Directory where the files are extracted
extract_dir = 'nfl_data'

# List of CSV files (you can modify this based on your actual files)
csv_files = [f for f in os.listdir(extract_dir) if f.endswith('.csv')]

# Read all CSV files into a dictionary of DataFrames
dfs = {file: pd.read_csv(os.path.join(extract_dir, file)) for file in csv_files}

# Access specific DataFrames
#for file_name, df in dfs.items():
print(f"Data from {'games.csv'}:")
    
# Specify the full path to the games.csv file
df = pd.read_csv(os.path.join(extract_dir, 'games.csv'))
print(df.head(1))  # Display first few rows of each DataFrame


Data from games.csv:
       gameId  season  week  gameDate gameTimeEastern homeTeamAbbr  \
0  2022090800    2022     1  9/8/2022        20:20:00           LA   

  visitorTeamAbbr  homeFinalScore  visitorFinalScore  
0             BUF              10                 31  


In [8]:
#rename column names- make them lower case and replace space or separate words with underscore
df.columns=df.columns.str.lower()
df.columns=df.columns.str.replace(' ','_')
print(df.head(10))

       gameid  season  week   gamedate gametimeeastern hometeamabbr  \
0  2022090800    2022     1   9/8/2022        20:20:00           LA   
1  2022091100    2022     1  9/11/2022        13:00:00          ATL   
2  2022091101    2022     1  9/11/2022        13:00:00          CAR   
3  2022091102    2022     1  9/11/2022        13:00:00          CHI   
4  2022091103    2022     1  9/11/2022        13:00:00          CIN   
5  2022091104    2022     1  9/11/2022        13:00:00          DET   
6  2022091105    2022     1  9/11/2022        13:00:00          HOU   
7  2022091106    2022     1  9/11/2022        13:00:00          MIA   
8  2022091107    2022     1  9/11/2022        13:00:00          NYJ   
9  2022091109    2022     1  9/11/2022        13:00:00          WAS   

  visitorteamabbr  homefinalscore  visitorfinalscore  
0             BUF              10                 31  
1              NO              26                 27  
2             CLE              24                 26 

In [9]:
#Load the data into sql server using replace option
import pandas as pd
import mysql.connector

# Define the append_to_sql function
def append_to_sql(df, table_name, connection):
    """
    Appends data from a DataFrame to a MySQL table.
    
    Parameters:
        df (pd.DataFrame): The DataFrame to append.
        table_name (str): The name of the target table in the database.
        connection (mysql.connector.connection_cext.CMySQLConnection): The MySQL connection object.
    """
    # Generate column placeholders dynamically based on DataFrame columns
    columns = ', '.join(df.columns)
    placeholders = ', '.join(['%s'] * len(df.columns))
    
    # Construct the INSERT query
    insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
    
    # Convert DataFrame to list of tuples for bulk insertion
    values = [tuple(row) for row in df.to_numpy()]
    
    # Execute the query
    cursor = connection.cursor()
    cursor.executemany(insert_query, values)
    connection.commit()
    print(f"Data appended successfully to {table_name}!")


In [ ]:
# Example Usage

# Create a sample DataFrame
data = {'id': [3, 4], 'name': ['Charlie', 'David']}
df = pd.DataFrame(data)

# Establish MySQL connection
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Asc90937866!',
    database='MySQLMusa'
)

# Call the function to append data
append_to_sql(df, 'df_orders', conn)

# Close the connection
conn.close()